In [11]:
#defining the libraries and the environment variables
import os
from dotenv import load_dotenv
import openai
import pandas as pd
import numpy as np
import re
import random

load_dotenv()

API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = API_KEY

pd.set_option('display.max_colwidth', None)

In [2]:
#define role
attribute_list = ["College student", "Graduate student", "College teacher"]
sex_list = ["male", "female"]
fisique_list = ["fat", "normal", "skinny"]


In [3]:
prompt="During a lunch break, find yourself in the college cafeteria. There, you'll see four distinct vegetable options to choose from. Your task is to select one, referring to it only by its number.\n\
When responding, please consider each option independently to ensure your selection is based solely on your preference."

In [4]:

choice_1 = "(1): Dynamite chili and tangy lime-seasoned beets\n\
            (2): Beets\n\
            (3): Lighter-choice beets with no added sugar\n\
            (4): High-antioxidant beets"

choice_2 = "(1): Beets\n\
            (2): Dynamite chili and tangy lime-seasoned beets\n\
            (3): Lighter-choice beets with no added sugar\n\
            (4): High-antioxidant beets"


choice_3 = "(1): High-antioxidant beets\n\
            (2): Beets\n\
            (3): Lighter-choice beets with no added sugar\n\
            (4): Dynamite chili and tangy lime-seasoned beets"

In [7]:
def generate_completion(role, choice, prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo-2024-04-09",
        #model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": f"{prompt},{choice}"}
        ],
        temperature=0.7,
        max_tokens=100,
        )
    return response

In [9]:
def ask_vegetable_preference(choice,prompt,range_number):
    df = pd.DataFrame(columns=["attribute","response","number"])
    
    for i in range(range_number):
        #attribute = "college student"
        attribute = random.choice(attribute_list)
        sex = random.choice(sex_list)
        fisique = random.choice(fisique_list)

        role = f"you are {attribute}."
        #you have to behave like {attribute}."

        response = generate_completion(role, choice, prompt)
        content = response.choices[0].message.content

        #extract number from responce
        numbers = re.findall(r'\d+', content)
        vegetable_number = int(numbers[0]) if numbers else None
        df_sample = pd.DataFrame({"attribute":attribute,"response":content,"number":vegetable_number},index=[0])

        df = pd.concat([df,df_sample],ignore_index=True)
    return df

In [12]:
df_1 = ask_vegetable_preference(choice_1,prompt,50)
df_2 = ask_vegetable_preference(choice_2,prompt,50)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define dictionaries to map numeric values to descriptive labels
choice_1_dic = {
    1: "Dynamite chili and \n tangy lime-seasoned beets",
    2: "Beets",
    3: "Lighter-choice beets \n with no added sugar",
    4: "High-antioxidant beets"
}

choice_2_dic = {
    1:"Beets",
    2:"Dynamite chili and \n tangy lime-seasoned beets",
    3:"Lighter-choice beets \n with no added sugar",
    4:"High-antioxidant beets"
}


choice_3_dic = {
    1:"Higher-antioxidant beets",
    2:"Beets",
    3:"Lighter-choice beets \n with no added sugar",
    4:"Dynamite chili and \n tangy lime-seasoned beets"
}

column = "number"

# Define a function to plot frequency of numbers associated with descriptive labels
def plot_frequency(df,column,choice_dic, ax):
    frequency = df[column].value_counts().sort_index()
    frequency.index = frequency.index.map(lambda x: choice_dic[x])
    desired_order = list(choice_1_dic.values())
    frequency = frequency.reindex(desired_order)
    frequency.plot(kind='bar', ax=ax)
    ax.set_xlabel('label')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Frequency of {column}')
    ax.tick_params(axis='x', rotation=0)

# Create DataFrame 1 (df_1) and DataFrame 2 (df_2) before calling this function

# Plot frequency for DataFrame 1
plt.figure(figsize=(15, 6))
ax1 = plt.subplot(1, 3, 1)
plot_frequency(df_1, column,choice_1_dic, ax1)

# Plot frequency for DataFrame 2
ax2 = plt.subplot(1, 3, 2)
plot_frequency(df_2, column,choice_2_dic, ax2)

#ax3 = plt.subplot(1, 3, 3)
#plot_frequency(df_3, column,choice_3_dic, ax3)

plt.tight_layout()
plt.show()
